In [1]:
# Edited example of keras-team mnist example of how to use the confusion matrix callback

'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import LambdaCallback


from sklearn.metrics import classification_report, confusion_matrix


Using TensorFlow backend.


In [4]:
batch_size = 128
num_classes = 10
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
def calculate_label_confusion(matrix,label):
    tp = matrix[:,label,label]
    fn = matrix[:,label,:].sum(axis=1) - tp
    fp = matrix[:,:,label].sum(axis=1) - tp
    tn = matrix[:,:,:].sum(axis=(1,2)) - (fn+fp+tp)
    return tp,tn,fp,fn

def precision(tp,tn,fp,fn):
    return tp/(tp+fp)

def recall(tp,tn,fp,fn):
    return tp/(tp+fn)

def accuracy(tp,tn,fp,fn):
    return (tp+tn)/(tp+tn+fp+fn)


In [0]:
def confusion_matrix_cell(true_class, pred_class):
    """
    Creates a metric that collects the value for a single cells of the confusion matrix
    :param true_class:
    :param pred_class:
    :return: Keras metric
    """

    def confusion(y_true, y_pred):
        """
        Collects the samples predicted as pred_class where its true class is true_class
        :param y_true:
        :param y_pred:
        :return: the number of predictions as mentioned above
        """
        # Calculate the label from one-hot encoding
        pred_class_label = K.argmax(y_pred, axis=-1)
        true_class_label = K.argmax(y_true, axis=-1)

        # Create a mask representing where the prediction is pred_class and the true class is true_class
        pred_mask = K.equal(pred_class_label, pred_class)
        true_mask = K.equal(true_class_label, true_class)
        mask = tf.logical_and(pred_mask, true_mask)

        # Get the total number of occurences
        occurrences = K.sum(K.cast(mask, 'int32'), axis=0)
        return occurrences

    return confusion

In [0]:
from keras.callbacks import Callback
class ConfusionMatrix(Callback):
    """
    A callback used to collect confusion matrix cell values.
    Is appropriate for multi-class problems.
    """
    def __init__(self,num_labels,matrix_saver=None,val_matrix_saver=None):
        """
        Initializes the callback
        :param num_labels: The number of classes
        :param matrix_saver: Should save the confusion matrix for each epoch
        """
        super(ConfusionMatrix, self).__init__()
        self.num_labels = num_labels
        self.matrix_saver = matrix_saver
        self.val_matrix_saver = val_matrix_saver
        self.metric_prefix = 'confusion_'
        self.val_metric_prefix = 'val_confusion_'

    def on_epoch_begin(self, epoch, logs=None):
        """
        Reset confusion matrix
        :param epoch:
        :param logs:
        :return:
        """
        self.matrix = np.zeros((self.num_labels,self.num_labels))

    def on_epoch_end(self, epoch, logs=None):
        """
        Saves confusion matrix and validation confusion matrix.
        IMPORTANT NOTE: Notice that the last batch in the validation dataset may cause
        wrong statistics (since keras returns the average)
        :param epoch:
        :param logs:
        :return:
        """
        #Save the confusion matrix for training.
        if self.matrix_saver is not None:
            self.matrix_saver(self.matrix/self.matrix.sum())

        # Saves the confusion matrix for validation.
        if self.val_matrix_saver is not None:
            self.matrix = np.zeros((self.num_labels,self.num_labels))
            self.collect_values(logs,self.val_metric_prefix)
            self.val_matrix_saver(self.matrix/self.matrix.sum())

    def on_batch_end(self, batch, logs=None):
        # Collect value for each of the confusion matrix cells
        self.collect_values(logs,self.metric_prefix)

    def collect_values(self,logs,metric_prefix):
        """
        Collects all the values for the confusion matrix
        :param logs:
        :param metric_prefix:
        :return:
        """
        for i in range(self.num_labels):
            for j in range(self.num_labels):
                self.collect_value(logs,metric_prefix,i,j)

    def collect_value(self,logs,metric_prefix,i,j):
        """
        Collects from logs the value for cell (i,j) in the confusion matrix
        :param logs: Keras logs dictionary object
        :param metric_prefix: The prefix of the metric collected up to the cell number
        :param i: true class
        :param j: predicted class
        :return:
        """
        metric_name = metric_prefix + str(i*self.num_labels + j + 1)
        self.matrix[i,j] += logs[metric_name]

    def generate_metrics(self):
        """
        Creates all metrics needed for creating the confusion matrix
        :return: A list of metrics representing the confusion matrix cells
        """
        return [confusion_matrix_cell(i, j) for i in range(self.num_labels) for j in range(self.num_labels)]


In [0]:

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

batch_print_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: print(logs))

'''
my_confusion = confusion_matrix(10,print,print)
confusion_metrics = my_confusion.generate_metrics()
'''

my_confusion = ConfusionMatrix(10,print,print)
confusion_metrics = my_confusion.generate_metrics()


In [14]:

type(my_confusion)


__main__.ConfusionMatrix

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']+confusion_metrics)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=[batch_print_callback,my_confusion],
          verbose=0)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: ignored